In [1]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
URL_DZEN_NEWS = 'https://dzen.ru/news/'
URL_LENTA_NEWS = 'https://lenta.ru/'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

params = {
    'sso_failed': '',
}

In [4]:
def get_content_dom_from_html_text(url, headers=None, params=None):
    response = requests.get(url, headers=headers, params=params)
    content_dom = html.fromstring(response.text)

    print(response.url)
    
    return content_dom

def parser_dzen_news(content_dom):
    news_container = content_dom.xpath("//div[contains(@class,'mg-card_flexible')]")
    dzen_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//span[@class='mg-card-source__source']/a/text()")[0]
        new_text = new_container.xpath(".//h2/a/text()")[0].replace('\xa0', ' ')
        new_link = new_container.xpath(".//h2/a/@href")[0]
        new_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        dzen_news.append(news_dict) 

    return dzen_news


def parser_lenta_news(content_dom):
    news_container = content_dom.xpath("//a[contains(@class, 'card-')]")
    lenta_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//*[name()='svg']/*[name()='use']/attribute::*")
        if len(new_source) == 1:
            new_source = new_source[0].split('ui-label_')
            if len(new_source) == 2:
                new_source = new_source[1]
            else:
                new_source = 'Lenta.ru'
        else:
            new_source = 'Lenta.ru' 
        new_text = new_container.xpath(".//span[contains(@class, 'card-')]/text()")
        if len(new_text) == 1:
            new_text = new_text[0]
        else:
            new_text = None  
        new_link = URL_LENTA_NEWS + new_container.xpath("./@href")[0]
        
        new_date = new_container.xpath(".//time/text()")
        if len(new_date) == 1:
            new_date = new_date[0]
        else:
            new_date = None
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        lenta_news.append(news_dict)
        
    return lenta_news



def get_news(content_dom, parser_name='dzen'):
    result = None
    
    if parser_name == 'dzen':
      result = parser_dzen_news(content_dom)
    elif parser_name == 'lenta':
      result = parser_lenta_news(content_dom)
    
    return result

In [5]:
dom = get_content_dom_from_html_text(URL_DZEN_NEWS, headers=headers, params=params)

https://dzen.ru/news?issue_tld=ru&sso_failed=


In [6]:
dzen_news = get_news(dom, 'dzen')
dzen_news

[{'new_source': 'РИА Новости',
  'new_text': 'Президент Путин назвал нападение украинских диверсантов в Брянской области терактом',
  'new_link': 'https://dzen.ru/news/story/Prezident_Putin_nazval_napadenie_ukrainskikh_diversantov_vBryanskoj_oblasti_teraktom--3763da8e12dc97e0aabb3264e5bd5c3a?lang=ru&rubric=index&fan=1&stid=zgJWnYolmuMDV9BqQU3a&t=1677770308&tt=true&persistent_id=2227760566&story=d09ac91d-fdc2-5542-9eee-258b50493aa2&issue_tld=ru',
  'new_date': '07:52'},
 {'new_source': 'РИА Новости',
  'new_text': 'Глава МИД России Лавров извинился перед Индией за поведение западных делегаций на G20',
  'new_link': 'https://dzen.ru/news/story/Glava_MID_Rossii_Lavrov_izvinilsya_peredIndiej_zapovedenie_zapadnykh_delegacij_naG20--a1c4eeacae05317e7dc69180de15ecc3?lang=ru&rubric=index&fan=1&stid=5g3meQFgZrZoW8z_f4hP&t=1677770308&tt=true&persistent_id=2220961058&story=06271c56-a8cd-5255-8689-38a658afe9e7&issue_tld=ru',
  'new_date': '07:13'},
 {'new_source': 'ТАСС',
  'new_text': 'Захарова: Р

In [7]:
pd.DataFrame(dzen_news)

,new_source,new_text,new_link,new_date
0,РИА Новости,Президент Путин назвал нападение украинских ди...,https://dzen.ru/news/story/Prezident_Putin_naz...,07:52
1,РИА Новости,Глава МИД России Лавров извинился перед Индией...,https://dzen.ru/news/story/Glava_MID_Rossii_La...,07:13
2,ТАСС,Захарова: РФ глубоко озабочена информацией о я...,https://dzen.ru/news/story/Zakharova_RF_glubok...,08:10
3,РИА Новости,В Молдавии оппозиция блокировала трибуну парла...,https://dzen.ru/news/story/VMoldavii_oppoziciy...,08:11
4,РИА Новости,Зампред ЦБ Белов: модернизированные банкноты в...,https://dzen.ru/news/story/Zampred_CB_Belov_mo...,08:12
...,...,...,...,...
60,ТАСС,Собянин сообщил о старте дилерских продаж авто...,https://dzen.ru/news/story/Sobyanin_soobshhil_...,07:59
61,Автостат,Кроссовер OMODA C5 получил полноприводную верс...,https://dzen.ru/news/story/Krossover_OMODA_C5_...,08:12
62,Lenta.ru,«Автостат»: продажи новых легковых автомобилей...,https://dzen.ru/news/story/Avtostat_prodazhi_n...,07:01
63,Автостат,АВТОВАЗ с 1 марта 2023 года повысил цены на ав...,https://dzen.ru/news/story/AVTOVAZ_s1_marta_20...,07:33


In [8]:
dom, response = get_content_dom_from_html_text(URL_LENTA_NEWS, headers=headers)

https://lenta.ru/


In [9]:
lenta_news = get_news(dom, 'lenta')
lenta_news

[{'new_source': 'Lenta.ru',
  'new_text': None,
  'new_link': 'https://lenta.ru//news/2023/03/02/bog_vyr/',
  'new_date': '17:33'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Россияне оказались заблокированы в тундре из-за метели на два дня',
  'new_link': 'https://lenta.ru//news/2023/03/02/saved/',
  'new_date': '18:15'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Стилист Рогов назвал вернувшиеся в моду тренды 1990-х годов',
  'new_link': 'https://lenta.ru//news/2023/03/02/returntrends/',
  'new_date': '18:11'},
 {'new_source': 'Lenta.ru',
  'new_text': 'В Германии раскритиковали требование Бербок к Лаврову',
  'new_link': 'https://lenta.ru//news/2023/03/02/lavrov_welt/',
  'new_date': '18:11'},
 {'new_source': 'Lenta.ru',
  'new_text': 'В Эстонии заявили о необходимости передать ядерное оружие Украине',
  'new_link': 'https://lenta.ru//news/2023/03/02/estua/',
  'new_date': '18:09'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Утопившего приятеля в реке россиянина приговорили к восьми